Installing and Importing Necessary Libraries: 

In [ ]:
!pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 stable-baselines gym-anytrading gym
!pip install quantstats
!pip install finta
!pip install ta

In [ ]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from finta import TA
from ta.utils import dropna
from ta.trend import MACD

# Reinforcement Learning
import gym
import gym_anytrading
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C,DQN
from gym_anytrading.envs import StocksEnv


#Decision Tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.metrics import classification_report

#Lstm network
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

Pre-Processing Data:

In [ ]:
def dataload_rl_tree(path):
  
  #Getting the data
  df = pd.read_csv(path)
  df['Date'] = pd.to_datetime(df['Date'])
  df.sort_values('Date', ascending=True, inplace=True)
  df['Volume'] = df['Volume'].apply(lambda x: float(x))
  df.set_index('Date', inplace=True)
  df=df.dropna()

  #Adding the technical Indicators
  
  df['EMA20'] = TA.EMA(df, 20)
  df['EMA50'] = TA.EMA(df, 50)
  df['EMA200']=TA.EMA(df,200)
  df['ATR']=TA.ATR(df)
  df['ADX'] = TA.ADX(df, 14)
  df['RSI'] = TA.RSI(df,14)
  df['OBV']=TA.OBV(df)
  macd=MACD(df['Close'])
  df['MACD']=macd.macd()
  df['MACDsignal']=macd.macd_signal()

  df.fillna(0, inplace=True)
  
  df['ClgtEMA200'] = np.where(df['Close'] > df['EMA200'], 1, -1)
  df['EMA20gtEMA50'] = np.where(df['EMA20'] > df['EMA50'], 1, -1)
  df['EMA50gtEMA200'] = np.where(df['EMA50'] > df['EMA200'], 1, -1)
  df['MACDSIGgtMACD'] = np.where(df['MACDsignal'] > df['MACD'], 1, -1)

  return df



In [ ]:
path='/content/drive/MyDrive/stockmarket/RELIANCE.NS.csv'

df_rl=dataload_rl_tree(path)

df_tree=dataload_rl_tree(path)

df_lstm=pd.read_csv(path,header=0,usecols=['Date','Close'],parse_dates=True,index_col='Date')
df_lstm=df_lstm.dropna()

df_ensemble=df_lstm

df_rl.tail()
df_tree.tail()
df_lstm.tail()

Reinforcement Learning:

In [ ]:
#Making a custom trading environment to include Technical Indicators

def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['EMA50gtEMA200','RSI', 'ADX','ATR','MACDSIGgtMACD','ClgtEMA200','EMA20gtEMA50','OBV']].to_numpy()[start:end]
    return prices, signal_features

In [ ]:
train_length=int(len(df_rl)*0.7)
print(train_length)
print(len(df_rl))

In [ ]:
class MyCustomEnv(StocksEnv):
    _process_data = add_signals
    
env2 = MyCustomEnv(df=df_rl, window_size=10, frame_bound=(10,train_length))

In [ ]:
# Making the dummy enivornment 

env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
# Training the model 

model = A2C('MlpLstmPolicy', env, verbose=1,tensorboard_log='/content/drive/MyDrive/stockmarket/Tensorboard/A2C') 
model.learn(total_timesteps=1000000)

In [ ]:
%load_ext tensorboard

In [ ]:
#To get the tensorboard log

%tensorboard --logdir '/content/drive/MyDrive/stockmarket/Tensorboard/A2C/A2C_1'

In [ ]:
#Remove the comment to save the model

#model.save('/content/drive/MyDrive/stockmarket/a2c.h5')

In [ ]:
# Remove the comment to load a previous trained model

#model.load('/content/drive/MyDrive/stockmarket/A2C_weights.h5',env)

In [ ]:
# Evaluating the results and storing the predicition from the model 

start=train_length-1
end=len(df_rl)

env = MyCustomEnv(df=df_rl, window_size=10, frame_bound=(start,end))

actions=[]

obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs) 
    actions.append(action)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break
        

In [ ]:
df_ensemble['Rl_pred']=np.NaN
df_ensemble.iloc[(len(df_ensemble) - len(actions)):,-1:] = actions

trade_dataset=df_ensemble.dropna()
trade_dataset['Rl_pred'].iloc[-1]=0
print(trade_dataset)

Decision Tree:

In [ ]:
#Normalizing the data and creating the target class

df_tree['target_cls'] = np.where(df_tree['Close'].shift(-1) > df_tree['Close'], 1, 0)

df_tree['ADX']=df_tree['ADX']/df_tree['ADX'].max()
df_tree['EMA20']=df_tree['EMA20']/df_tree['EMA20'].max()
df_tree['EMA50']=df_tree['EMA50']/df_tree['EMA50'].max()
df_tree['EMA200']=df_tree['EMA200']/df_tree['EMA200'].max()
df_tree['ATR']=df_tree['ATR']/df_tree['ATR'].max()

df_tree.head()

In [ ]:
#Definint the predictor Variables

predictors_list = ['EMA200','ADX','EMA50','ATR','EMA20']
X = df_tree[predictors_list]
X.tail()

In [ ]:
#Splitting the data into training and testing

y_cls = df_tree.target_cls
y_cls.tail()


y=y_cls
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X, y, test_size=0.3, random_state=432 ,stratify=y)

print (X_cls_train.shape, y_cls_train.shape)
print (X_cls_test.shape, y_cls_test.shape)

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
def returns(df,clf,y_cls_pred):
  df['y_cls_pred'] = np.NaN
  df.iloc[(len(df) - len(y_cls_pred)):,-1:] = y_cls_pred
  trade_dataset = df.dropna()
  trade_dataset['Tomorrow_pred'] = trade_dataset['y_cls_pred'].shift(-1)
  trade_dataset.fillna(0,inplace=True)

  num_share=0
  money=1000000
  init_money=money

  for day, row in trade_dataset.iterrows():
    if (row['Tomorrow_pred']==1):
      if (money>row['Close']):
        num_share+=1
        money-=(row['Close'])

    elif (row['Tomorrow_pred']==0 and num_share>0):
      money+=num_share*(row['Close'])
      num_share=0

  if (num_share>0):
    print(left)
  change=float(((money-init_money)/init_money)*100)
  return change


In [ ]:
#To find the best model configuration

max=0.0
final_leaf=0
final_depth=0

for depth in range(3,20):

    for leaf in range(3,10):
      clf = DecisionTreeClassifier(criterion='entropy',max_depth=depth, min_samples_leaf=leaf)
      clf = clf.fit(X_cls_train, y_cls_train)
      y_cls_pred = clf.predict(X_cls_test)
      change=returns(df_tree,clf,y_cls_pred)

      if (change>max):
        max=change
        final_leaf=leaf
        final_depth=depth
        print('===============',depth)
        print('==',leaf)
        print(max)

In [ ]:
#Using the obtained depth and leaf to make the decision tree

print(final_depth,final_leaf)
clf = DecisionTreeClassifier(criterion='entropy',max_depth=final_depth,min_samples_leaf=final_leaf)
clf = clf.fit(X_cls_train, y_cls_train)
y_cls_pred = clf.predict(X_cls_test)
report = classification_report(y_cls_test, y_cls_pred)
print(report)

In [ ]:
#Visualizing the tree

dot_data = tree.export_graphviz(clf, out_file=None,filled=True,feature_names=predictors_list)
graphviz.Source(dot_data)
graphviz.Source(dot_data).format='png'
graphviz.Source(dot_data).render('dtree_render',view=True)

In [ ]:
#Storing the decision tree predictions

trade_dataset['Tree_pred']=y_cls_pred
trade_dataset['Tree_pred'].iloc[-1]=0
print(trade_dataset)

Lstm Network: 

In [ ]:
path='/content/drive/MyDrive/stockmarket/RELIANCE.NS.csv'
df_lstm=pd.read_csv(path,header=0,usecols=['Date','Close'],parse_dates=True,index_col='Date')
df_lstm=df_lstm.dropna()

In [ ]:
#Normalizing the data

scaler=MinMaxScaler()
scaled_data=scaler.fit_transform (df_lstm)
print(scaled_data)

In [ ]:
#Spliting the data into training and testing

train_length=int(len(scaled_data)*0.7)
test_length=len(scaled_data)-train_length
train_data=scaled_data[0:train_length,:]
test_data=scaled_data[train_length:len(scaled_data),:]

print(test_length)

In [ ]:
def create_dataset(dataset,timestep=1):
  data_x,data_y=[],[]
  for i in range(len(dataset)-timestep-1):
    data_x.append(dataset[i:(i+timestep),0])
    data_y.append(dataset[i+timestep,0])

  return np.array(data_x),np.array(data_y)


In [ ]:
timestep=10
train_x,train_y=create_dataset(train_data,timestep)
test_x,test_y=create_dataset(test_data,timestep)
print(test_x.shape)

In [ ]:
train_x=np.reshape(train_x,(train_x.shape[0],1,train_x.shape[1]))
test_x=np.reshape(test_x,(test_x.shape[0],1,test_x.shape[1]))

In [ ]:
#Defining the model architecture 

model=Sequential()

model.add(LSTM(256,return_sequences = True,input_shape=(1,timestep)))
model.add(Dropout(0.2))
model.add(LSTM(units = 128, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 256))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_x,train_y,epochs=1,batch_size=1,verbose=1)
score=model.evaluate(train_x,train_y,verbose=0)

print('Loss=',score[0])
print('Accuracy=',score[1])

In [ ]:
#Getting the prediction from the model

train_predict=model.predict(train_x)
test_predict=model.predict(test_x)

In [ ]:
#Converting to unnormalized data

train_predict=scaler.inverse_transform(train_predict)
train_y=scaler.inverse_transform([train_y])

test_predict=scaler.inverse_transform(test_predict)
test_y=scaler.inverse_transform([test_y])

In [ ]:
print(test_predict)

In [ ]:
#Storing the prediction of the lstm network

trade_dataset['price_pred']=np.NaN
trade_dataset.iloc[(len(trade_dataset) - len(test_predict)):,-1:] = test_predict
trade_dataset=trade_dataset.dropna()
trade_dataset['Lstm_pred']=np.where(trade_dataset['price_pred'].shift(-1) > trade_dataset['price_pred'], 1, 0)
trade_dataset['Tree_pred'].iloc[-1]=0

print(trade_dataset)

In [ ]:
train_predict_plot=np.empty_like(scaled_data)
train_predict_plot[:,:]=np.nan
train_predict_plot[1:len(train_predict)+1,:]=train_predict

test_predict_plot=np.empty_like(scaled_data)
test_predict_plot[:,:]=np.nan
test_predict_plot[len(train_predict)+1+(timestep*2):len(scaled_data)-1,:]=test_predict

In [ ]:
#Obtaining the graph of actual stock price vs predicited price 

plt.figure(figsize=(15,5))
plt.plot(scaler.inverse_transform(scaled_data),label='Actual Stock Price')
plt.plot(train_predict_plot, label='Price Predicted during Traning')
plt.plot(test_predict_plot,label='Price Predicited during Testing')
plt.xlabel("Day",fontsize=15)
plt.ylabel("Price",fontsize=15)
plt.legend(framealpha=1, frameon=True)
plt.savefig('/content/drive/MyDrive/stockmarket/figures/lstm_prediction.jpeg')
plt.show()


Ensemble Learning:

In [ ]:
#Pre processing the data for ensemble learning

trade_dataset['Rl_pred'] = trade_dataset['Rl_pred'].apply(lambda x: int(x))

print(trade_dataset)

In [ ]:
#Taking max voting from all the model and storing it

from statistics import mode

trade_dataset['Final_pred']=np.NaN

for day,row in trade_dataset.iterrows():
  trade_dataset.at[day,'Final_pred']=mode([row['Rl_pred'],row['Tree_pred'],row['Lstm_pred']])

print(trade_dataset)

In [ ]:
#Function to calculate the result and obtaining the graphs

def trade_result(name,method):

  num_share=0
  money=100000
  init_money=money

  td=trade_dataset
  td['buy']=0
  returns=[]
  last_value=init_money

  for day, row in trade_dataset.iterrows():
    if (row[method]==1):
      if (money>row['Close']):
        num_share+=1
        money-=(row['Close'])
        td.at[day,'buy']=1

    elif (row[method]==0 and num_share>0):
      money+=num_share*(row['Close'])
      td.at[day,'buy']=-1
      num_share=0
      last_value=money

    returns.append(last_value)

  change=float(((money-init_money)/init_money)*100)
  profit=money-init_money

  td['returns']=returns

  path1='/content/drive/MyDrive/stockmarket/figures/'+name+'_buy.png'
  path2='/content/drive/MyDrive/stockmarket/figures/'+name+'_portfolio.png'


  plt.figure(figsize=(15,10))
  plt.plot(td['Close'])
  plt.scatter(td.loc[td['buy'] ==1].index,td.loc[td['buy'] ==1, 'Close'].values, label='skitscat', color='green', s=25, marker="^")
  plt.scatter(td.loc[td['buy'] ==-1].index,td.loc[td['buy'] ==-1, 'Close'].values, label='skitscat', color='red', s=25, marker="v")
  plt.title(name+": Total Profit: %.6f" % profit + ' ~ ' +
              "Percentage Change: %.6f" % change,fontsize=15)
  plt.xlabel("Date",fontsize=15)
  plt.ylabel("Price",fontsize=15)
  
  plt.savefig(path1)

  plt.figure(figsize=(15,10))
  plt.plot(td['returns'])
  plt.title( name+": Portfolio Value Over Time",fontsize=15)
  plt.xlabel("Date",fontsize=15)
  plt.ylabel("Money",fontsize=15)

  plt.savefig(path2)





In [ ]:
#Obtaining Results from all the model 

trade_result('Reinforcement Learning','Rl_pred')
trade_result('Decision Tree','Tree_pred')
trade_result('LSTM Network','Lstm_pred')
trade_result('Ensemble Learning','Final_pred')